In [15]:
pip install MFDFA

Note: you may need to restart the kernel to use updated packages.


In [16]:
from MFDFA import MFDFA
#import csv
import pyedflib
import glob as gl
import matplotlib.image as mpimg
from skimage.transform import resize
#import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import neighbors
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn import svm, metrics
from sklearn.model_selection import GridSearchCV

import numpy as np
from numpy.fft import fft, fftfreq
from scipy import signal
import matplotlib.pyplot as plt
from mne.time_frequency.tfr import morlet
from mne.viz import plot_filter, plot_ideal_filter
import mne


In [14]:
def filtroFIR(sig,fp):
    sfreq = 256
    f_p = fp
    transition_band = 0.25 * f_p
    f_s = f_p + transition_band


    h = mne.filter.create_filter(sig, sfreq, l_freq=None, h_freq=f_p,
                                 fir_design='firwin2', verbose=True)
    sig_filtrada = np.convolve(h, sig)[len(h) // 2:]
    
    return sig_filtrada


In [4]:
# %%
# %%
# %%
import mne

sample_dir = r"D:\Usuario\OneDrive\Escritorio\Uni\Pasantia\PacienteCon10.EDF"


f = pyedflib.EdfReader(sample_dir )
n = f.signals_in_file

signal_labels = f.getSignalLabels()
sigbufs = np.zeros((17, f.getNSamples()[0]))
sigbufs[0, :] = (f.readSignal(0)-f.readSignal(1)) #Fp1-F3
sigbufs[1, :] = (f.readSignal(1)-f.readSignal(2)) #F3-C3
sigbufs[2, :] =(f.readSignal(2)-f.readSignal(3)) #C3-P3
sigbufs[3, :] =(f.readSignal(3)-f.readSignal(4)) #P3-O1
sigbufs[4, :] =(f.readSignal(0)-f.readSignal(5)) #Fp1-F7
sigbufs[5, :] =(f.readSignal(5)-f.readSignal(6)) #F7-T3
sigbufs[6, :] =(f.readSignal(6)-f.readSignal(7)) #T3-T5
sigbufs[7, :] =(f.readSignal(7)-f.readSignal(4)) #T5-O1
sigbufs[8, :] =(f.readSignal(19)) #Cz-Ref
sigbufs[9, :] =(f.readSignal(9)-f.readSignal(10)) #Fp2-F4
sigbufs[10, :] =(f.readSignal(10)-f.readSignal(11)) #F4-C4
sigbufs[11, :] =(f.readSignal(11)-f.readSignal(12)) #C4-P4
sigbufs[12, :] =(f.readSignal(12)-f.readSignal(13)) #P4-O2
sigbufs[13, :] =(f.readSignal(9)-f.readSignal(14)) #Fp2-F8
sigbufs[14, :] =(f.readSignal(14)-f.readSignal(15)) #F8-T4
sigbufs[15, :] =(f.readSignal(15)-f.readSignal(16)) #T4-T6
sigbufs[16, :] =(f.readSignal(16)-f.readSignal(4)) #T6-O1



In [12]:
sig = []
fpL =30

timeI = 80  
timeF = 100 

inicio = int(timeI*256)
final = int(timeF*256)

for i in range(0,17):
    
    sig.append(filtroFIR(sigbufs[i][inicio:final],fpL))
    
    
sig =np.array(sig)


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 sec)

Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 sec)

Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6

In [9]:
import numpy as np

sfreq = 256  # Frecuencia de muestreo


ch_names = ['Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 
            'Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1','Cz-Ref' ,'Fp2-F4', 
            'F4-C4', 'C4-P4', 'P4-O2', 'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O1']

info = mne.create_info(ch_names=ch_names, sfreq=sfreq)

raw = mne.io.RawArray(sigbufs[0:17][:], info)

scalings = {'mag': 2, 'grad': 2}

raw.plot(n_channels=4, scalings=scalings, title='Data from arrays',
         show=True, block=True)



Creating RawArray with float64 data, n_channels=17, n_times=389120
    Range : 0 ... 389119 =      0.000 ...  1519.996 secs
Ready.
Channels marked as bad:
none


In [13]:

sfreq = 256  # Sampling frequency




ch_names = ['Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 
            'Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1','Cz-Ref' ,'Fp2-F4', 
            'F4-C4', 'C4-P4', 'P4-O2', 'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O1']

info = mne.create_info(ch_names=ch_names, sfreq=sfreq)

raw = mne.io.RawArray(sig[0:17][:], info)

# Scaling of the figure.
# For actual EEG/MEG data different scaling factors should be used.
scalings = {'mag': 2, 'grad': 2}


raw.plot(n_channels=4, scalings=scalings, title='Data from arrays',
         show=True, block=True)

Creating RawArray with float64 data, n_channels=17, n_times=5233
    Range : 0 ... 5232 =      0.000 ...    20.438 secs
Ready.
Channels marked as bad:
none
